In [51]:
import pandas as pd
import os
import pathlib as Path
import matplotlib.pyplot as plt
import glob
import ast
import re

In [11]:

# Path to your results folder
path = os.path.join('..', 'results')  # This navigates from notebooks to results folder

# Get all CSV files in the folder
all_files = glob.glob(os.path.join(path, "*.csv"))

# Create an empty list to store individual dataframes
dfs = []

# Read each CSV file and append to the list
for file in all_files:
    df_temp = pd.read_csv(file)
    # Optional: Add a column to track which file the data came from
    df_temp['source_file'] = os.path.basename(file)
    dfs.append(df_temp)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Check the result
print(f"Total number of files processed: {len(all_files)}")
print(f"Shape of combined DataFrame: {combined_df.shape}")
print("\nColumns in combined DataFrame:")
print(combined_df.columns.tolist())

Total number of files processed: 49
Shape of combined DataFrame: (397495, 17)

Columns in combined DataFrame:
['person_id', 'name', 'gender', 'religion_ethnicity', 'social_status_job', 'role_in_case', 'titles', 'date', 'calendar', 'place_name', 'place_type', 'legal_case_type', 'case_result', 'case_result_type', 'row_index', 'case_unique_id', 'source_file']


In [23]:
result = combined_df['source_file'].unique()
result = list(result)
len(result)

49

In [24]:
combined_df['gender'].value_counts()

gender
Man      232512
Woman     22949
Name: count, dtype: int64

In [25]:
combined_df['legal_case_type'].value_counts()

legal_case_type
Property Disputes               6178
Debt/Loan Agreements            5448
Inheritance Cases               4421
Slavery and Manumission         2389
Criminal Cases                  2142
                                ... 
Lease Agreement                    1
Guardianship and supervision       1
Tebennî                            1
Gift                               1
Medical Liability                  1
Name: count, Length: 73, dtype: int64

In [25]:
combined_df['legal_case_type'].value_counts()

legal_case_type
Property Disputes               6178
Debt/Loan Agreements            5448
Inheritance Cases               4421
Slavery and Manumission         2389
Criminal Cases                  2142
                                ... 
Lease Agreement                    1
Guardianship and supervision       1
Tebennî                            1
Gift                               1
Medical Liability                  1
Name: count, Length: 73, dtype: int64

In [27]:
pd.set_option('display.max_rows', None)

In [50]:
social_status_counts = combined_df['social_status_job'].value_counts()

In [30]:
combined_df.head()

,person_id,name,gender,religion_ethnicity,social_status_job,role_in_case,titles,date,calendar,place_name,place_type,legal_case_type,case_result,case_result_type,row_index,case_unique_id,source_file
0,beşiktaş mahkemesi_2_hüküm no: 1_1,Mehmed Bey b. Abdullah,Man,Muslim,NaN,Borrower,['Bey'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
1,beşiktaş mahkemesi_2_hüküm no: 1_2,Keyvan Kethüdâ b. Abdurrahman,Man,Muslim,NaN,Buyer,['Kethüdâ'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
2,beşiktaş mahkemesi_2_hüküm no: 1_3,Devlet Hatun,Woman,Muslim,NaN,Neighbor,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
3,beşiktaş mahkemesi_2_hüküm no: 1_4,Muharrem Usta,Man,Muslim,Artisan/Craftsperson,Neighbor,['Usta'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
4,beşiktaş mahkemesi_2_hüküm no: 1_5,Pîr b. Hürrem,Man,Muslim,NaN,Neighbor,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv


In [39]:
#Unique titles
unique_titles = []
for titles_str in combined_df['titles'].dropna():
    titles = ast.literal_eval(titles_str)
    unique_titles.extend(titles)
unique_titles = sorted(set(unique_titles))

In [42]:
len(unique_titles)

5211

In [55]:
def normalize_characters(text):
    # Dictionary of character replacements
    replacements = {
        'â': 'a', 'ā': 'a', 'á': 'a', 'à': 'a',
        'î': 'i', 'ī': 'i', 'í': 'i', 'ì': 'i',
        'û': 'u', 'ū': 'u', 'ú': 'u', 'ù': 'u',
        'ô': 'o', 'ō': 'o', 'ó': 'o', 'ò': 'o',
        'ê': 'e', 'ē': 'e', 'é': 'e', 'è': 'e',
        'ḫ': 'h',
        'ḥ': 'h',
        'ṣ': 's',
        'ṭ': 't'
    }
    
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

def clean_title_advanced(title):
    # Initial basic cleaning
    title = title.lower().strip()
    title = normalize_characters(title)

    # Remove common variations and standardize
    # Remove 'el-' or 'el ' prefix which is common in Arabic names
    if title.startswith('el-') or title.startswith('el '):
        title = title[3:]
    
    # Remove parentheses and their contents
    title = re.sub(r'\(.*?\)', '', title)
    
    # Remove numbers
    title = re.sub(r'\d+', '', title)
    
    # Final cleanup of any extra spaces
    title = ' '.join(title.split())
    
    return title

# Apply advanced cleaning
cleaned_titles_advanced = [clean_title_advanced(title) for title in unique_titles]
cleaned_unique_titles_advanced = sorted(set(cleaned_titles_advanced))

print(f"Original number of titles: {len(unique_titles)}")
print(f"Number of titles after advanced cleaning: {len(cleaned_unique_titles_advanced)}")

Original number of titles: 5211
Number of titles after advanced cleaning: 3984


In [66]:
print(f"Number of unique social status values: {len(combined_df['social_status_job'].unique())}")

Number of unique social status values: 1078


In [1]:
print(combined_df['social_status_job'].value_counts(dropna=True).sort_index())

NameError: name 'combined_df' is not defined

In [76]:
categories = {
    'Sufi': ['Sufi', 'Sufi Leader', 'Derviş', 'Şeyh', 'Dede', 'Baba', 'Mevlevi', 'Bektaşi', 'Nakşibendi', 'Halveti'],
    
    'Religious': ['Scholar/Ulama', 'Imam', 'Müezzin', 'Religious Leader', 'Clergy', 'Priest', 'Bishop', 'Metropolitan', 'Hatib', 'Vaiz'],
    
    'Military': ['Janissary', 'Soldier', 'Military', 'Sipahi', 'Commander', 'Military Officer', 'Military Official', 'Cündî', 'Serdengeçdi'],
    
    'Administrative': ['Bureaucrat', 'Judge', 'Scribe', 'Tax Collector', 'Governor', 'Trustee', 'Kethüda', 'Steward', 'Kadı', 'Nâib', 'Mütevellî'
                      'Treasurer'],
    
    'Commercial': ['Trader/Merchant', 'Money Changer', 'Sarraf', 'Merchant', 'Bakkal', 'Attâr'],
    
    'Artisanal': ['Artisan/Craftsperson', 'Carpenter', 'Blacksmith', 'Baker', 'Butcher', 'Tailor', 'Debbağ', 'Sarrâc'],
    
    'Service': ['Servant', 'Porter', 'Guard', 'Doorman', 'Gatekeeper', 'Cook', 'Bevvâb', 'Hamal', 'Tabbâh'],
    
    'Maritime': ['Sailor', 'Ship Captain', 'Captain', 'Boatman', 'Reis', 'Kayıkçı'],
    
    'Agricultural': ['Farmer/Agriculturist', 'Shepherd', 'Gardener', 'Bostancı'],
    
    'Professional': ['Physician/Healer', 'Architect', 'Engineer', 'Teacher', 'Translator', 'Muallim'],
    
    'Social Status': ['Slave/Enslaved Person', 'Freed Slave', 'Sultan', 'Royalty', 'Minor', 'Orphan', 'Müdebber']
}

In [77]:
# Create new column with lowercase values first
combined_df['cleaned_social_status'] = combined_df['social_status_job'].str.lower()

In [ ]:
def standardize_social_status(value):
    if pd.isna(value):
        return value
    
    # Convert to lowercase
    value = value.lower().strip()
    
    # Standardize specific cases
    standardization_map = {
        'acemi oğlan': 'military',
        'acemi oğlanı': 'military',
        'acemioğlan': 'military',
        'acemioğlanı': 'military',
        '40th yaya bölüğü yaya': 'military',
        'abacı': 'artisanal',
        'accountant': 'administrative',
        'accountant of waqfs' : 'administrative',
        'agent' : 'administrative',
        'akideci (trader/merchant)' : 'artisanal',
        'ambassador' : 'administrative',
        'amil' : 'religious',
        'anadolu ağası' : 'military',
        'apprentice' : 'artisanal',
        'arab ağa' : 'military',
        'arabacı' : 'artisanal',
        'architect' : 'professional',
        'armaşi' : 'administrative',
        'arpa emîni' : 'administrative',
        'arpacı' : 'artisanal',
        'artisan/craftsperson' : 'artisanal',
        'artisan/craftsperson (grocer)' : 'artisanal',
        'artisan/craftsperson (perfumer)' : 'artisanal',
        'ases': 'military',
        'asesbaşı' : 'military',
        'asesbaşı (chief of night watchmen)': 'military',
        'asesler kethüdâsı' : 'military',
        'askerî kassâm' : 'military',
        'assistant' : 'administrative',
        'attar' : 'artisanal',
        'attâr' : 'artisanal',
        'auctioneer' : 'artisanal',
        'azeb' : 'military',
        'ağa' : 'military',
        'ağa of the saadetlu kaymakam pasha' : 'military',
        'aşçı' : 'service',
        'aşçı-i sultânî' : 'service',
        'aşçıbaşı' : 'service',
        'bahriye kâtibi' : 'administrative',
        'bahçevân' : 'service',
        'bakara-hân' : 'religious',
        'baker' : 'artisanal',
        'bakkal' : 'artisanal',
        'bakkal (grocer)' : 'artisanal',
        'bakkalân pazarbaşı' : 'artisanal',
        'baltacı' : 'artisanal',
        'baltacılar kethudası' : 'artisanal',
        'ban' : 'artisanal',
        'barber' : 'artisanal',
        'barber and grocer' : 'artisanal',
        'bathhouse attendant' : 'artisanal',
        'bathhouse keeper' : 'artisanal',
        'bayrakdâr' : 'military',
        'bayraktar' : 'military',
        'baytar' : 'service',
        'baş defterdâr' : 'administrative',
        'başbâkīkulu' : 'administrative',
        'başkorucu' : 'military',
        'baştezkireci' : 'military',
        'başyedekçi' : 'military',
        'bekçi' : 'military',
        'bevvâb' : 'military',
        'bevvâb (doorkeeper)' : 'military',
        'bevvâb (gatekeeper)' : 'military',
        'bevvâb-ı sultanî' : 'military',
        'bevvâb-ı sultanî (gatekeeper)' : 'military',
        'bevvâb-ı sultânî' : 'military',
        'bevvâbü dârü’s-saâde' : 'military',
        'bevvâbü’s-sultânî' : 'military',
        'bey' : 'administrative',
        'bey of antakya' : 'administrative',
        'beylerbeyi' : 'administrative',
        'beytülmal emîni' : 'administrative',
        'beytülmal officer' : 'administrative',
        'beytülmal-i hassa emîni' : 'administrative',
        'beytülmâl' : 'administrative',
        'beytülmâl emin' : 'administrative',
        'beytülmâl emîni' : 'administrative',
        'beytülmâl emîni (treasurer)' : 'administrative',
        'beytülmâl emîni (treasury official)' : 'administrative',
        'beytülmâl emîni (treasury trustee)' : 'administrative',
        'beytülmâl mültezimi' : 'administrative',
        'beytülmâl-i hâs emîni' : 'administrative',
        'beytülmâl-i hâssı emîni' : 'administrative',
        'beytülmâl-i mezbûrun nâzırı' : 'administrative',
        'beytülmâl-i âmme emîni' : 'administrative',
        'beytülmâl-i âmmın emîni' : 'administrative',
        'beytü’l-mâl emîni' : 'administrative',
        'beytü’l-mâl officer' : 'administrative',
        'bezci' : 'artisanal',
        'bezzâzistân kethüdâsı' : 'artisanal',
        'bishop' : 'religious',
        'blacksmith' : 'artisanal',
        'boatman' : 'artisanal',
        'boatmen guild leader' : 'artisanal',
        'bookbinder' : 'artisanal',
        'bostancı' : 'military',
        'bostancı kethüdâsı' : 'military',
        'bostancıbaşı' : 'military',
        'bostancılar kethüdâsı başhasekisi' : 'military',
        'bostancılar odabaşısı' : 'military',
        'bostânî' : 'military',
        'bostânî (gardener)' : 'military',
        'bozacı' : 'artisanal',
        'bureaucrat' : 'administrative',
        'bureaucrat (former judge)' : 'administrative',
        'bursa kadısı' : 'administrative',
        'butcher' : 'artisanal',
        'butcher guild steward' : 'artisanal',
        'button maker' : 'artisanal',
        'buzcu' : 'artisanal',
        'bâb-ı sa‘âdet bevvâb' : 'military',
        'bölükbaşı' : 'military',
        'börekçi' : 'artisanal',
        'cabi' : 'administrative',
        'calligrapher' : 'artisanal',
        'camel driver' : 'artisanal',
        'candle maker guild steward' : 'artisanal',
        'captain' : 'military',
        'caretaker' : 'artisanal',
        'carpenter' : 'artisanal',
        'carriage driver' : 'artisanal',
        'cebeci' : 'military',
        'cebecibaşı' : 'military',
        'celeb tâ’ifesinden' : 'artisanal',
        'cerrâh (surgeon)' : 'artisanal',
        'chef' : 'artisanal',
        'chief butler' : 'artisanal',
        'chief clerk' : 'administrative',
        'chief of bailiffs' : 'administrative',
        'chief usher' : 'administrative',
        'city officer' : 'administrative',
        'cizye officer' : 'administrative',
        'clergy' : 'religious',
        'clergyman' : 'religious',
        'cleric' : 'administrative',
        'clockmaker' : 'artisanal',
        'collector' : 'administrative',
        'commander' : 'military',
        'community leader' : 'administrative',
        'convert' : 'religious',
        'cook' : 'artisanal',
        'cook at the new imperial palace' : 'artisanal',
        'cotton carder' : 'artisanal',
        'court clerk' : 'administrative',
        'court messenger' : 'administrative',
        'court officer' : 'administrative',                                     
        'court official' : 'administrative',                                     
        'court scribe' : 'administrative', 
        'court servant' : 'administrative', 
        'court summoner' : 'administrative',                     
        'court trustee' : 'administrative', 
        'crafts guild member' : 'artisanal',
        'crafts guild member (carpenter)' : 'artisanal',
        'craftsman' : 'artisanal',
        'craftsperson' : 'artisanal', 
        'crier' : 'artisanal',        
        'current sâhib-i ayâr' : 'administrative',
        'customs officer' : 'administrative',
        'customs official' : 'administrative',
        'câbi' : 'administrative',
        'câbi (tax collector)' : 'administrative',
        'câbi and kāimmakām-ı mütevellî' : 'administrative',
        'câbî' : 'administrative',
        'câbî (collector)' : 'administrative',
        'câbî and kāimmakām-ı mütevellî' : 'administrative',
        'câbî-i evkāf' : 'administrative',
        'câbî-i kârbânsaray' : 'administrative',
        'câbî-i vakf' : 'administrative',
        'câbî-i âsiyâb' : 'administrative',
        'cündi' : 'military',                                             
        'cündî' : 'military',                                            
        'cündî (military personnel)' : 'military',                         
        'cündî (soldier)' : 'military',                                  
        'cüzhân' : 'artisanal',                                          
        'danişmend' : 'military',                                         
        'davud paşa mütevellîsi' : 'administrative',                             
        'debbağ' : 'artisanal',                                             
        'debbağ (tanner)' : 'artisanal',                                   
        'defterdar' : 'administrative',                                          
        'defterdâr' : 'administrative',                                          
        'dellâl' : 'administrative',                                             
        'dellâl (auctioneer)' : 'administrative',                                
        'demircibaşı' : 'artisanal',                                        
        'deputy' : 'religious',                                             
        'deputy imam' : 'religious',                                        
        'deputy imam and teacher' : 'religious',                           
        'deputy judge' : 'administrative',                                       
        'deputy trustee' : 'administrative',                                     
        'dergâh-ı âlî solak': 'sufi',                                 
        'dergâh-ı âlî çorbacı' : 'artisanal',                               
        'derviş' : 'sufi',                                             
        'devir-hân' : 'sufi',                                          
        'diplomat/envoy' : 'administrative',                                    
        'divan kâtibi' : 'administrative',                                       
        'doorkeeper' : 'artisanal',                                         
        'doorman' : 'artisanal',                                           
        'driver' : 'artisanal',                                             
        'dârü’s-sa‘âde ağa' : 'administrative',                                  
        'dârü’ş-şifâ ağa' : 'administrative',                                    
        'dîvân efendisi' : 'administrative',                                     
        'dîvân-ı âlî kâtibi' : 'administrative',                                 
        'düğmeci' : 'artisanal',                                            
        'eczâ-hân' : 'artisanal',                                           
        'eczâ-hân (pharmacist)' : 'artisanal',                             
        'eflak voyvoda' : 'administrative',                                      
        'ehl-i hibre' : 'artisanal',                                        
        'ekmekci' : 'artisanal',                                            
        'ekmekçiler kethüdâsı' : 'artisanal',                               
        'el-bevvâb' : 'artisanal',                                          
        'el-câbî' : 'artisanal',                                            
        'el-cündî' : 'artisanal',                                           
        'el-kâtib' : 'administrative',                                           
        'el-muhzır' : 'administrative',                                         
        'el-mübâşir' : 'administrative',                                         
        'el-mülâzım' : 'administrative',                                         
        'el-münâdî' : 'administrative',                                          
        'elder' : 'administrative',                                              
        'emekdar' : 'artisanal',                                           
        'emin' : 'administrative',                                               
        'emin-i beytü’l-mâl' : 'administrative',                                 
        'emir-i ahur' : 'military',                                        
        'emir-i alay' : 'military',                                        
        'emânet-i beytü’l-mâl zâbiti' : 'administrative',                        
        'emîn' : 'administrative',                                               
        'emîn-i beytülmâl' : 'administrative',                                   
        'emîn-i beytülmâl-i hâs' : 'administrative',                             
        'emîn-i darbhâne' : 'administrative',                                    
        'emîn-i defterî' : 'administrative',                                     
        'emîn-i matbah-ı mahmud paşa' : 'administrative',                        
        'emîni and mültezimi' : 'administrative',                                
        'engineer' : 'artisanal',                                           
        'enslaved person' : 'slavery',                                   
        'enslaved person (formerly)' : 'slavery',                         
        'erbâb-ı timar' : 'administrative',                                      
        'erbâb-ı tımar' : 'administrative',                                      
        'es-silâhî' : 'military',                                          
        'esirci (slave trader)' : 'slavery',                              
        'esîrci' : 'slavery',                                             
        'executor' : 'administrative',                                           
        'executor of will' : 'administrative',                                   
        'expert witness' : 'administrative',                                     
        'ez-zevvâk' : 'artisanal',   
        'fahrü’l-a‘yân' : 'administrative',                                      
        'fahrü’l-cuyûş' : 'military',                                      
        'fahrü’l-cüyûş' : 'military',                                      
        'fahrü’l-eimme' : 'administrative',                                      
        'fahrü’l-kuzât' : 'administrative',                                      
        'fahrü’l-küttâb' : 'administrative',                                     
        'fahrü’l-muhadderât' : 'administrative',                                 
        'fahrü’l-müderrisîn' : 'religious',                                 
        'fahrü’l-ümenâ' : 'administrative',                                      
        'fahrü’l-ümerâ' : 'administrative',                                      
        'fahrü’ş-şüyûh' : 'religious',                                      
        'farm manager' : 'farming',                                       
        'farmer' : 'farming',                                              
        'farmer/agriculturist' : 'farming',                             
        'ferrâş' : 'artisanal',                                             
        'ferrâş (caretaker)' : 'artisanal',                                 
        'ferrâş (caretaker/sweeper)' : 'artisanal',                         
        'fethiye i̇mamı' : 'religious',                                     
        'fisherman' : 'artisanal',                                          
        'flower seller' : 'artisanal',                                      
        'foot soldier' : 'military',                                       
        'former anatolian kadıaskeri' : 'administrative',                        
        'former bey of peçuy' : 'administrative',                                
        'former beytü’l-mâl emîni' : 'administrative',                          
        'former bosnian judge' : 'administrative',                               
        'former bostancıbaşı' : 'military',                                
        'former collector' : 'administrative',                                   
        'former customs officer' : 'administrative',                             
        'former fish commissioner' : 'administrative',                           
        'former grocer' : 'artisanal',                                      
        'former imam' : 'religious',                                        
        'former inspector' : 'administrative',                                   
        'former janissary' : 'military',                                   
        'former janissary kethüda' : 'military',                           
        'former judge' : 'administrative',                                       
        'former judge of aleppo' : 'administrative',                             
        'former judge of baghdad' : 'administrative',                            
        'former judge of damascus' : 'administrative',                           
        'former judge of ebâ eyyûb-i ensârî' : 'administrative',                 
        'former judge of erzurum' : 'administrative',                            
        'former judge of kuds-i şerîf' : 'administrative',                       
        'former judge of mahmiye-i galata' : 'administrative',                   
        'former judge of manisa' : 'administrative',                             
        'former judge of medina' : 'administrative',                             
        'former judge of mekke-i mükerreme' : 'administrative',                  
        'former judge of tavas' : 'administrative',                              
        'former kadı' : 'administrative',                                        
        'former kadı of yenişehir' : 'administrative',                           
        'former maraş kadı' : 'administrative',                                  
        'former military judge' : 'administrative',                              
        'former mütesellim' : 'administrative',                                  
        'former qadi of jerusalem' : 'administrative',                          
        'former rumeli kazasker' : 'administrative',                             
        'former ship captain' : 'military',                                
        'former slave' : 'slavery',                                      
        'former sofya mütesellim' : 'administrative',                            
        'former sürekçi, current meyhâneci' : 'artisanal',                  
        'former tokat judge' : 'administrative',                                 
        'former trustee' : 'administrative',                                    
        'former turnacıbaşı' : 'artisanal',                                 
        'former voivode of boğdan' : 'administrative',                           
        'former voivode of moldavia' : 'administrative',                         
        'former waqf trustee' : 'administrative',                                
        'formerly enslaved person' : 'slavery',                           
        'freed person' : 'slavery',                                       
        'freed slave' : 'slavery',                                       
        'freedman' : 'slavery',                                           
        'freedwoman' : 'slavery',                                         
        'furrier' : 'artisanal',
        'Galata Crier' : 'artisanal',
        'Galata voyvodası' : 'artisanal',                             
        'Gardener' : 'artisanal',                                    
        'Gatekeeper' : 'artisanal',                               
        'Gemi Reisi' : 'artisanal',                                 
        'Governor' : 'administrative',                              
        'Grocer' : 'artisanal',                           
        'Guard' : 'military',                                              
        'Guardian' : 'military',                                           
        'Guardian of Orphans' : 'military',                                
        'Gümrük Emîni' : 'artisanal',                         
        'Gümrük Hamalı' : 'artisanal',                        
        'Gılmân-ı Acemiyân' : 'slavery',                        
        'Haffâf' : 'artisanal',                     
        'Hafizu’l-kitâb' : 'religious',                       
        'Hafız' : 'religious',                   
        'Hafız (Guardian of the Book)' : 'administrative',                  
        'Hafız-ı Kitâb Yaya' : 'administrative',                 
        'Hafızu’l-kitâb' : 'administrative',                  
        'Hafızü’l-kitâb' : 'administrative',                
        'Halife' : 'religious',                  
        'Halife-i Mekteb' : 'religious',                 
        'Halîfe-i Mekteb' : 'religious',                
        'Halîfe-i mekteb' : 'religious',               
        'Halîfe-i rûznâmçe-i küçük' : 'administrative',             
        'Hamal' : 'artisanal',            
        'Hamal (Porter)' : 'artisanal',             
        'Hamal Kethüdâsı' : 'artisanal',             
        'Hamallar' : 'artisanal',            
        'Hamam Keeper' : 'artisanal',             
        'Hamam Operator' : 'artisanal',             
        'Hamamcı' : 'artisanal',
        'Hamamcı-i Ayasofya' : 'artisanal',                                 
        'Hamamcı-i Mehmed Paşa der-Galata' : 'artisanal',                   
        'Hamamcı-i Çinili' : 'artisanal',                                   
        'Hamamcılar Kethüdâsı' : 'artisanal',                               
        'Hamlacı' : 'artisanal',                                            
        'Hammal' : 'artisanal',                                             
        'Hammam Keeper' : 'artisanal',                                      
        'Hammam operator' : 'artisanal',                                    
        'Hammâmî' : 'artisanal',                                           
        'Hammâmî-i Aksaray' : 'artisanal',                                  
        'Hammâmî-i Piyale Paşa' : 'artisanal',                              
        'Hammâr (Wine Seller)' : 'artisanal',                               
        'Harbendebaşı' : 'artisanal',                                       
        'Haremeyn Mütevellîsi' : 'administrative',                               
        'Haremeyn mütevellîsi' : 'administrative',                               
        'Hasbî mütevellî' : 'administrative',                                    
        'Haseki Ağa' : 'administrative',                                         
        'Hasköy Subaşı' : 'military',                                      
        'Hassa Bostancı' : 'military',                                     
        'Hassa Cook' : 'artisanal',                                         
        'Hassa Mehter' : 'military',                                       
        'Hassa Tabbah' : 'artisanal',                                       
        'Hasırcı' : 'artisanal',                                            
        'Hatib' : 'religious',                                             
        'Hatip' : 'religious',                                              
        'Hatîb' : 'religious',                                             
        'Hatîb (Preacher)' : 'religious',                                   
        'Hatîb (preacher)' : 'religious',                                   
        'Hatîb-i Câmi‘-i Mehmed' : 'religious',                             
        'Hayyât-ı emîrî' : 'administrative',                                     
        'Hazinedâr Başı' : 'administrative',                                     
        'Hazînedar' : 'administrative',                                          
        'Hazînedâr' : 'administrative',                                          
        'Head Muezzin' : 'religious',                                       
        'Head of special division' : 'administrative',                           
        'Head tailor' : 'artisanal',                                        
        'Helvacı' : 'artisanal',                                            
        'Hoca-i mekteb' : 'religious',                                      
        'Hâfız' : 'religious',                                              
        'Hâfızu’l-kitâb' : 'administrative',                                     
        'Hâfızü’l-kitâb' : 'administrative',                                     
        'Hâmil-i hâze’l-kitâb' : 'administrative',                               
        'Hâssa Architect' : 'artisanal',                                    
        'Hâssa Bostancı' : 'military',                                     
        'Hâssa Koyun Sürücüsü' : 'artisanal',                               
        'Hâssa Sürücü' : 'artisanal',                                       
        'Hâssa architect' : 'artisanal',                                    
        'Hâssa bostancı' : 'military',                                     
        'Hâssa helvacı' : 'artisanal',                                      
        'Hâssa sarrâc' : 'artisanal',                                       
        'Hâssa su nâzırı' : 'artisanal',                                    
        'Hâssa tavukçularından' : 'artisanal',                              
        'Hâssa topçular zümresinden' : 'artisanal',                         
        'Hâssa çakırcısı' : 'artisanal',                                    
        'Hâzin-i Sultânî' : 'administrative',                                    
        'Imam' : 'religious',                                            
        'Imam and Hatib' : 'religious',                                     
        'Imam and Teacher' : 'religious',                                   
        'Imam and preacher' : 'religious',                                  
        'Imam of Istabl-ı âmire' : 'religious',                             
        'Imam, Scholar/Ulama' : 'religious',                                
        'Imâm' : 'religious',
        'Innkeeper' : 'artisanal',                                          
        'Inspector of Waqfs' : 'administrative',                                 
        'Interpreter' : 'administrative',                                        
        'Irgad' : 'artisanal',                                              
        'Irgat' : 'artisanal',                                              
        'Istari' : 'artisanal',                                             
        'Janissary' : 'military',                                       
        'Janissary Ağa' : 'military',                                      
        'Janissary Beytülmâl Emin' : 'military',                           
        'Janissary Beytülmâl Emîni' : 'military',                          
        'Janissary Commander' : 'military',                                
        'Janissary Scribe' : 'military',                                   
        'Janissary, Piyade Bey' : 'military',                              
        'Jeweler' : 'artisanal',                                            
        'Judge' : 'administrative',                                            
        'Judge (Kadı)' : 'administrative',                                       
        'Kadi' : 'administrative',                                               
        'Kadi (Judge)' : 'administrative',                                       
        'Kadı' : 'administrative',                                              
        'Kadı (Judge)' : 'administrative',                                       
        'Kadıasker-i Muhzır' : 'administrative',                                 
        'Kadırga Reisi (Galley Captain)' : 'military',                     
        'Kahveci' : 'artisanal',                                            
        'Kaimmakam' : 'administrative',                                          
        'Kalafatçı (Shipbuilder)' : 'artisanal',                            
        'Kalaycı' : 'artisanal',                                            
        'Kalyon Reis' : 'military',                                        
        'Kapudan' : 'military',                                            
        'Kapudân' : 'military',                                            
        'Kapıcı' : 'military',                                             
        'Kapıcıbaşı' : 'military',                                         
        'Kapıcılar Bölükbaşı' : 'military',                                
        'Kapıcılar Bölükbaşısı' : 'military',                              
        'Kapıcılar Kethudası' : 'military',                                
        'Kapıcılar Kethüdâsı' : 'military',                                
        'Karaman beylerbeyi' : 'administrative',                                 
        'Karcıbaşı' : 'artisanal',                                          
        'Kasapbaşı' : 'artisanal',                                          
        'Kassab' : 'artisanal',                                             
        'Kassam' : 'military',                                             
        'Kassâb' : 'artisanal',                                             
        'Kassâm' : 'military',                                             
        'Kassâm Kâtibi' : 'military',                                      
        'Kassâm-ı Askerî' : 'military',                                    
        'Kassâm-ı askerî' : 'military',                                    
        'Kassâm-ı askerî muhzırı' : 'military',                            
        'Kassâmü’l-askerî' : 'military',                                   
        'Katib' : 'administrative',                                              
        'Katib (Scribe)' : 'administrative',                                     
        'Kavukçular Kethüdâsı' : 'artisanal',                               
        'Kayyim' : 'administrative',                                             
        'Kayyum' : 'administrative',                                            
        'Kayyûm' : 'administrative',                                            
        'Kayyım' : 'administrative',                                            
        'Kayyım (Custodian)' : 'administrative',                                 
        'Kayyım-i câmi‘-i şerîf' : 'administrative',                             
        'Kayık Kapıcısı' : 'artisanal',                                     
        'Kayıkçı' : 'artisanal',                                            
        'Kayıkçılar Kethüdâsı' : 'artisanal',                               
        'Kazasker' : 'administrative',                                           
        'Kazganî' : 'artisanal',                                            
        'Kazzâz (Silk Weaver)' : 'artisanal',                               
        'Kehhâl (Oculist)' : 'artisanal',                                   
        'Kerasteci' : 'artisanal',                                          
        'Keresteci' : 'artisanal', 
        'Kethudâ' : 'artisanal',                                            
        'Kethudâ-yı Hazret-i Efendi' : 'artisanal',                         
        'Kethudâ-yı Silâhdârân' : 'artisanal',                              
        'Kethudâ-yı keresteciyân' : 'artisanal',                            
        'Kethudâ-yı muhzırân' : 'artisanal',                                
        'Kethudâyeri Kâtibi' : 'artisanal',                                 
        'Kethüda' : 'artisanal',                                           
        'Kethüda (Steward)' : 'artisanal',                                  
        'Kethüda of Bezzâzistân' : 'artisanal',                             
        'Kethüda-i Sipahiyan' : 'artisanal',                                
        'Kethüda-yı Kassâbân' : 'artisanal',                                
        'Kethüda-yı bezzâzistân' : 'artisanal',                             
        'Kethüda-yı haffâfân' : 'artisanal',                                
        'Kethüda-yı muhzırân' : 'artisanal',                                
        'Kethüda-yı muhzırîn' : 'artisanal',                                
        'Kethüda-yı pereme' : 'artisanal',                                  
        'Kethüdayeri' : 'artisanal',                                        
        'Kethüdâ' : 'artisanal',                                           
        'Kethüdâ-i Hammâmcıyân' : 'artisanal',                              
        'Kethüdâ-i Mimar' : 'artisanal',                                    
        'Kethüdâ-i teberdârân' : 'artisanal',                               
        'Kethüdâ-yı Azeb' : 'artisanal',                                    
        'Kethüdâ-yı Kürkçüyân' : 'artisanal',                               
        'Kethüdâ-yı Muhzırân' : 'artisanal',                                
        'Kethüdâ-yı Topçuyân' : 'artisanal',                                
        'Kethüdâ-yı bezzâzistan-ı Galata' : 'artisanal',                    
        'Kethüdâ-yı bostancıyân' : 'artisanal',                             
        'Kethüdâ-yı bostâniyân' : 'artisanal',                              
        'Kethüdâ-yı ekmekçiyân' : 'artisanal',                              
        'Kethüdâ-yı habbâzan' : 'artisanal',                               
        'Kethüdâ-yı haffâfân' : 'artisanal',                                
        'Kethüdâ-yı hamamciyân' : 'artisanal',                              
        'Kethüdâ-yı kasabân' : 'artisanal',                                 
        'Kethüdâ-yı kassâbân' : 'artisanal',                                
        'Kethüdâ-yı muhzırân' : 'artisanal',                               
        'Kethüdâ-yı muhzırîn' : 'artisanal',                                
        'Kethüdâ-yı mumcuyân' : 'artisanal',                                
        'Kethüdâ-yı çiftlik' : 'artisanal',                                 
        'Kethüdâyeri' : 'artisanal',                                        
        'Kethüdâyeri Çukadârı' : 'artisanal',                               
        'Kethüdâ’ü’l-muhzırîn' : 'artisanal',                               
        'Keyyâl-i Kapan' : 'artisanal',                                     
        'Khatib' : 'administrative',                                             
        'Kilarî' : 'artisanal',                                             
        'Kiler-i hâssa kethüdâsı' : 'artisanal',                            
        'Kilerci' : 'artisanal',                                            
        'Kilârî' : 'artisanal',                                             
        'Kolicari' : 'artisanal',                                           
        'Kolisi' : 'artisanal',                                             
        'Korucu' : 'military',                                             
        'Kutucu (Box Maker)' : 'artisanal',                                
        'Kuşçu' : 'artisanal',                                              
        'Kâid-i sultânî' : 'administrative',                                     
        'Kâimmakâm-ı Mütevellî' : 'administrative',                              
        'Kâtib' : 'administrative',                                             
        'Kâtib (Scribe)' : 'administrative',                                     
        'Kâtib (scribe)' : 'administrative',                                     
        'Kâtib-i Gümrük' : 'administrative',                                     
        'Kâtib-i Muhâsebe' : 'administrative',                                   
        'Kâtib-i Ruznâmçe (Scribe)' : 'administrative',                          
        'Kâtib-i Vakf-ı Kılıç Ali Paşa' : 'administrative',                      
        'Kâtib-i Vakıf (Scribe of the Waqf)' : 'administrative',                 
        'Kâtib-i evkāf-ı mezbûre' : 'administrative',                            
        'Kâtib-i gümrük' : 'administrative',                                     
        'Kâtib-i mehterân-ı hâssa' : 'administrative',                           
        'Kâtib-i müşâhere' : 'administrative',                                   
        'Kâtib-i vakf' : 'administrative',                                       
        'Kâtib-i vakıf' : 'administrative',                                      
        'Kâtibü’l-hurûf' : 'administrative',                                     
        'Kömürcü' : 'artisanal',                                            
        'Köprücü' : 'artisanal',                                            
        'Köşk bekçisi' : 'military',                                       
        'Kürekçi (Oarsman)' : 'artisanal',                                  
        'Kādî' : 'administrative',                                               
        'Kāid' : 'administrative',                                               
        'Kāim-i makām' : 'administrative',                                       
        'Kāimmakām-ı imâm' : 'administrative',    
        'Laborer' : 'artisanal',                                            
        'Legal Representative' : 'administrative',                               
        'Levend' : 'military',                                             
        'Logofen'  : 'administrative',                                            
        'Market Inspector' : 'administrative',                                   
        'Masraf Kâtibi' : 'administrative',                                      
        'Master Builder' : 'artisanal',                                     
        'Master Craftsman' : 'artisanal',                                   
        'Mefharü’l-vâ‘izîn' : 'administrative',                                  
        'Mehter' : 'military',                                             
        'Mehter (Musician)' : 'military',                                  
        'Mekârî' : 'artisanal',                                             
        'Mellâh' : 'artisanal',                                             
        'Mendilci' : 'artisanal',                                           
        'Menzil Kethudası' : 'artisanal',                                   
        'Meremmâtî' : 'artisanal',                                          
        'Metropolitan' : 'religious',                                      
        'Mevkūfâtî' : 'artisanal',                                          
        'Meyhaneci (Innkeeper)' : 'artisanal',                              
        'Meyhaneci (Tavern Keeper)' : 'artisanal',                          
        'Meyhâneci' : 'artisanal',                                          
        'Meyhâneci (Tavern Keeper)' : 'artisanal',                          
        'Military' : 'military',                                          
        'Military Accountant' : 'military',                                
        'Military Assessor' : 'military',                                  
        'Military Judge' : 'administrative',                                     
        'Military Judge (kassâm-ı askerî)' : 'administrative',                   
        'Military Officer' : 'military',                                  
        'Military Official' : 'military',                                 
        'Military Scribe' : 'administrative',                                    
        'Military official' : 'military',                                  
        'Milkman' : 'artisanal',                                            
        'Miller' : 'artisanal',                                             
        'Miner' : 'artisanal',                                              
        'Minor' : 'artisanal',                                              
        'Mirâhor' : 'artisanal',                                           
        'Mirâhur' : 'artisanal',                                           
        'Mirâhur-ı sâbıkā' : 'military',                                   
        'Mi‘mâr' : 'artisanal',                                             
        'Mi‘mâr-ı Haremeyn                                  
        'Money Changer' : 'artisanal',                                      
        'Monk' : 'religious',                                               
        'Mu\'taka (Freed Slave)' : 'slavery',    ##burasi patlayabilir                          
        'Muallim' : 'religious',                                            
        'Muallim (Teacher)' : 'religious',                                  
        'Muarrif' : 'religious',                                            
        'Mubashir' : 'administrative',                                           
        'Muezzin' : 'religious',                                           
        'Mufti' : 'religious',                                              
        'Muharrerü’l-hurûf' : 'religious',                                  
        'Muhtesib' : 'administrative',                                          
        'Muhtesib Ağa' : 'administrative',                                       
        'Muhtesib oğlanı' : 'administrative',                                    
        'Muhzır' : 'administrative',                                           
        'Muhzır (Court Officer)' : 'administrative',                             
        'Muhzır (Court Official)' : 'administrative',                            
        'Muhzırbaşı' : 'administrative',                                         
        'Mukaddem' : 'administrative',                                           
        'Mukayyid' : 'administrative',                                           
        'Mukâtaacı' : 'administrative',                                          
        'Mukāta‘acı' : 'administrative',                                         
        'Mumcu' : 'artisanal',                                              
        'Mumcular Kethüdâsı' : 'artisanal',                                 
        'Musician' : 'artisanal',                                           
        'Musketeer' : 'artisanal',                                          
        'Mutawalli (Trustee)' : 'administrative',                                
        'Mutemed' : 'administrative',                                            
        'Mu‘allim' : 'religious',                                           
        'Mu‘allim-i sıbyân' : 'religious',                                  
        'Mu‘arrif' : 'religious',                                           
        'Mu‘temed' : 'artisanal',                                           
        'Mu‘îd' : 'artisanal',                                              
        'Mîr-i kıbtiyân' : 'administrative',                                     
        'Mîr-âhûr' : 'artisanal',                                           
        'Mîrî Sürücü' : 'artisanal',                                        
        'Mîrî Yelkenci' : 'artisanal',                                      
        'Mübaşir' : 'administrative',                                            
        'Mübâşir' : 'administrative',                                           
        'Mücellid' : 'artisanal',                                           
        'Müdebber' : 'slavery',                                           
        'Müdebbere (Slave/Enslaved Person)' : 'slavery',                  
        'Müderris' : 'religious',                                          
        'Müderris (Teacher)' : 'religious',                                 
        'Müderris (teacher)' : 'religious',                                 
        'Müezzin' : 'religious',                                         
        'Müezzin (Caller to Prayer)' : 'religious',                         
        'Müezzin and Cüzhân' : 'religious',                                 
        'Müezzin and Kayyım' : 'religious',                                 
        'Müezzin-i Samanveren' : 'religious',                               
        'Müezzin?' : 'religious',                                           
        'Mühtedi' : 'religious',                                            
        'Mühürdâr' : 'administrative',                                           
        'Mülâzım' : 'artisanal',                                            
        'Müneccimbaşı (Chief Astrologer)' : 'artisanal',                    
        'Münâdî' : 'religious',                                            
        'Müsellem' : 'religious',                                           
        'Müste‘id' : 'artisanal',                                           
        'Müteferrika' : 'administrative',                                       
        'Müteferrika-i sultânî' : 'administrative',                              
        'Mütesellim' : 'administrative',                                         
        'Mütevelli' : 'administrative',                                         
        'Mütevelli (Trustee)' : 'administrative',                                
        'Mütevellî' : 'administrative',                                         
        'Mütevellî (Former Trustee)' : 'administrative',                         
        'Mütevellî (Trustee)' : 'administrative',                               
        'Mütevellî-i Haremeyi şerîfeyn' : 'administrative',
        'Nakkaş' : 'artisanal',                                             
        'Nakkāş' : 'artisanal',                                             
        'Nakīb-i Hammâmcıyân' : 'artisanal',                                
        'Nalband' : 'artisanal',                                            
        'Navy Commander' : 'military',                                     
        'Na‘â' : 'artisanal',                                              
        'Neighborhood Imam' : 'religious',                                  
        'New Muslim' : 'religious',                                         
        'Nişancı Hamamcısı' : 'artisanal',                                  
        'Nişastacılar Kethüdâsı' : 'artisanal',                            
        'Noktacı' : 'artisanal',                                            
        'Nâib' : 'administrative',                                               
        'Nâib (Deputy)' : 'administrative',                                      
        'Nâib-i Recül' : 'administrative',                                       
        'Nâib-i kapan' : 'administrative',                                       
        'Nâib-i recül' : 'administrative',                                       
        'Nâibü’ş-şer‘' : 'administrative',                                       
        'Nâibü’ş-şer‘ (Deputy Judge)' : 'administrative',                        
        'Nâibü’ş-şer‘i’ş-şerîf' : 'administrative',                              
        'Nâzır' : 'administrative',                                             
        'Nâzır (Overseer)' : 'administrative',                                   
        'Nüvvâb-ı Sultânî' : 'administrative',
        'Odabaşı' : 'administrative',                                           
        'Officer' : 'administrative',                                            
        'Ordu-yı hümâyûn berberi' : 'artisanal',                            
        'Orphan' : 'religious',                                             
        'Otakçılar kethüdâsı' : 'artisanal',                                
        'Overseer' : 'administrative',                                           
        'Overseer of Darbhâne-i âmire' : 'administrative',                       
        'Overseer of the Waqf' : 'administrative',                               
        'Overseer of the mentioned waqf' : 'administrative',                     
        'Overseer of the waqf' : 'administrative',                               
        'Painter' : 'artisanal',                                            
        'Palace Gatekeeper' : 'military',                                  
        'Patriarch' : 'religious',                                          
        'Pencik Emîni' : 'artisanal',                                       
        'Peremeci' : 'artisanal',                                           
        'Peremeciler Kethudâsı' : 'artisanal',                              
        'Perfumer' : 'artisanal',                                           
        'Peyk-i hâssa' : 'military',                                       
        'Pharmacist' : 'artisanal',                                         
        'Physician' : 'administrative',                                          
        'Physician/Healer' : 'administrative',                                 
        'Pilgrim' : 'religious',                                          
        'Pirinçcibaşı' : 'artisanal',                                       
        'Poet/Writer' : 'religious',                                        
        'Porter' : 'artisanal',                                             
        'Posnifos' : 'religious',                                           
        'Postancı' : 'military',                                           
        'Preacher' : 'religious',                                           
        'Priest' : 'religious',                                            
        'Prophet' : 'religious', 
        'Qadi' : 'administrative',                                               
        'Ra\'iyyet': 'administrative',                                           
        'Raht Hazinedarı' : 'administrative',                                    
        'Reis' : 'military',                                             
        'Reis (Captain)' : 'military',                                     
        'Reis (Ship Captain)' : 'military',                                
        'Reis-i Asesân' : 'military',                                      
        'Reis-i Bölük' : 'military',                                       
        'Reis-i Habbâz (Head Baker)' : 'artisanal',                         
        'Reis-i Habbâzîn-i Sultânî' : 'artisanal',                          
        'Reis-i Sultanî' : 'artisanal',                                     
        'Reis-i Sultânî' : 'artisanal',                                    
        'Reis-i emîri' : 'artisanal',                                       
        'Reis-i muhzır' : 'artisanal',                                      
        'Reis-i muhzırân' : 'artisanal',                                    
        'Reis-i muhzırîn' : 'artisanal',                                    
        'Reis-i mücellidân' : 'artisanal',                                  
        'Reis-i müezzinîn' : 'artisanal',                                   
        'Reis-i nakkāş' : 'artisanal',                                      
        'Reis-i sağrıcıyân' : 'artisanal',                                  
        'Reis-i sultânî' : 'artisanal',                                    
        'Reisülküttâb' : 'administrative',                                       
        'Reisü’l-Muhzıran' : 'artisanal',                                   
        'Reisü’l-habbâzîn' : 'artisanal',                                   
        'Reisü’l-hayyat' : 'artisanal',                                     
        'Reisü’l-hayyâtîn (Head of Tailors)' : 'artisanal',                 
        'Reisü’l-küttâb' : 'artisanal',                                     
        'Reisü’l-muhzirîn' : 'artisanal',                                   
        'Reisü’l-muhzır' : 'artisanal',                                     
        'Reisü’l-muhzırîn' : 'artisanal',                                 
        'Reisü’l-muhzırîn (Chief of the Bailiffs)' : 'artisanal',           
        'Reisü’l-müezzinîn' : 'religious',                                  
        'Reisü’s-sultânî' : 'artisanal',                                   
        'Religious' : 'religious',                                          
        'Religious Leader' : 'religious',                                 
        'Religious Leader (Former Armenian Patriarch)' : 'religious',       
        'Religious Official' : 'religious',                                
        'Religious Scholar' : 'religious',                                  
        'Repairman' : 'artisanal',                                          
        'Ressam' : 'artisanal',                                             
        'Retired' : 'military',                                            
        'Retired Janissary' : 'military',                                  
        'Reîsü’l-habbâzîn' : 'artisanal',                                   
        'Reîsü’l-muhzirîn' : 'administrative',                                   
        'Reîsü’l-muhzır (Chief of Bailiffs)' : 'administrative',                 
        'Reîsü’l-muhzırîn' : 'administrative',                                   
        'Reîsü’t-tüccâr' : 'artisanal',                                     
        'Risale Ağa' : 'military',                                         
        'Rower' : 'artisanal',                                              
        'Royalty' : 'administrative',                                            
        'Rumeli Ağa' : 'administrative',                                         
        'Râcil' : 'artisanal',                                              
        'Râcil (Pedestrian)' : 'artisanal',                                 
        'Râhib (Monk)' : 'religious', 
        'Saddler' : 'artisanal',                                            
        'Sahhaf' : 'artisanal',                                             
        'Sahhâf' : 'artisanal',                                             
        'Sahib-i Timar' : 'administrative',                                      
        'Sailor' : 'artisanal',                                            
        'Sakabaşı' : 'artisanal',                                           
        'Sakalardan' : 'artisanal',                                         
        'Sakalık' : 'artisanal',                                            
        'Sakayân-ı Hâssa' : 'artisanal',                                    
        'Salt seller' : 'artisanal',                                        
        'Sancakbeyi' : 'administrative',                                         
        'Sandalcı' : 'artisanal',                                           
        'Sarac' : 'artisanal',                                              
        'Sarac-ı hâssa' : 'artisanal',                                      
        'Saray-ı atîk-i sultânî teberdâr' : 'military',                    
        'Sarrac-ı Sultânî (Saddler of the Sultan)' : 'artisanal',           
        'Sarraf' : 'artisanal',                                             
        'Sarraj' : 'artisanal',                                             
        'Sarraç (Saddler)' : 'artisanal',                                   
        'Sarrâc' : 'artisanal',                                            
        'Sarrâc-ı Hâssa' : 'artisanal',                                     
        'Sarrâc-ı Sultanî' : 'artisanal',                                   
        'Sarrâc-ı Sultânî' : 'artisanal',                                   
        'Sarrâc-ı Sultânî (Saddler)' : 'artisanal',                         
        'Sarrâc-ı Sultânî (Sultan\'s Saddler)' : 'artisanal',                
        'Sarrâc-ı emîr' : 'artisanal',                                      
        'Sarrâc-ı hâssa' : 'artisanal',                                     
        'Sarrâc-ı sultânî' : 'artisanal',                       
        'Sarrâclar Yiğidbaşısı' : 'artisanal',                              
        'Sarrâcü’s-sultânî' : 'artisanal',                                  
        'Sağ bölük ulûfecisi' : 'military',                                
        'Sağrıcı' : 'artisanal',                                            
        'Scholar' : 'religious',,                                           
        'Scholar/Ulama' : 'religious',                                  
        'Scribe' : 'administrative',                                           
        'Scribe of the Waqf' : 'administrative',                                 
        'Seafarer' : 'artisanal',                                           
        'Second-hand dealer' : 'artisanal',                                 
        'Sefîne Reis' : 'administrative',                                        
        'Sefîne Reisi' : 'administrative',                                       
        'Sekban' : 'military',                                             
        'Sekbân' : 'military',                                             
        'Selam Çavuşu' : 'military',                                       
        'Semân Medresesi Müderrisi' : 'scholar',                          
        'Ser-aşçıyân' : 'artisanal',                                        
        'Ser-bölük' : 'military',                                          
        'Ser-bölük (military officer)' : 'military',                       
        'Ser-bölük-i Teberdar' : 'military',                               
        'Ser-bölük-i muhzırân' : 'military',                               
        'Ser-bölük-i topçuyân' : 'military',                               
        'Ser-ferrâş' : 'artisanal',                                         
        'Ser-kasabân' : 'artisanal',                                        
        'Ser-kilârî' : 'artisanal',                                         
        'Ser-kuyumcu' : 'artisanal',                                        
        'Ser-mahfil' : 'artisanal',                                         
        'Ser-mehterân' : 'military',                                       
        'Ser-muhzır' : 'administrative',                                         
        'Ser-muhzırân' : 'administrative',                                       
        'Ser-oda' : 'military',                                            
        'Ser-çavuş' : 'military',                                          
        'Ser-çavuşân' : 'military',                                        
        'Serbölük' : 'military',                                           
        'Serdengeçdi Ağa' : 'military',                                    
        'Sergeant' : 'military',                                           
        'Serhammâlîn' : 'artisanal',                                        
        'Sermuhzırân' : 'administrative',                                        
        'Serrâc' : 'artisanal',                                             
        'Serrâcü’s-sultânî' : 'artisanal',                                  
        'Servant' : 'artisanal',                                           
        'Serçavuş' : 'military',                                           
        'Sheep Counter' : 'artisanal',                                      
        'Shepherd' : 'artisanal',                                          
        'Sherbet Seller' : 'artisanal',                                    
        'Ship Captain' : 'military',                                      
        'Silahdar' : 'military',                                           
        'Silahdâr' : 'military',                                          
        'Silahdârlar Cemâ‘ati Member' : 'military',                        
        'Silahdârân' : 'military',                                         
        'Silverhouse Worker' : 'artisanal',                                 
        'Silâhdâr' : 'military',                                           
        'Sipahi' : 'military',                                           
        'Sipâhiyân Kethüdâ Yeri' : 'military',                             
        'Sipâhî' : 'military',                                             
        'Slave' : 'slavery',                                              
        'Slave Owner' : 'slavery',                                        
        'Slave Trader' : 'slavery',                                      
        'Slave/Enslaved Person' : 'slavery',                           
        'Slave/Enslaved Person (Freed)' : 'slavery',                      
        'Slave/Enslaved Person, Artisan/Craftsperson' : 'slavery',        
        'Solak' : 'military',                                              
        'Solak Gurebâsı' : 'military',                                     
        'Solaklar kethüdâsı' : 'military',                                 
        'Soldier' : 'military',                                          
        'Standard Bearer' : 'military',                                    
        'Steward' : 'artisanal',                                            
        'Steward of tailors' : 'artisanal',                                 
        'Student' : 'religious',                                            
        'Subaşı' : 'military',                                            
        'Subaşı (Chief of Police)' : 'military',                           
        'Subaşı (local governor)' : 'military',                            
        'Subaşı (local official)' : 'military',                            
        'Subaşı Hasekisi' : 'military',                                    
        'Subaşı\'s agent' : 'military',                                     
        'Subaşı\'s man' : 'military',                                       
        'Sufi' : 'sufi',                                            
        'Sufi Leader' : 'sufi',                                       
        'Sultan' : 'administrative',                                            
        'Sultan\'s Doorkeeper' : 'military',                                
        'Sultan\'s Doorman' : 'military',                                   
        'Sultan\'s Gatekeeper' : 'military',                               
        'Sultan\'s Saddler' : 'artisanal',                                   
        'Sultan\'s gatekeeper' : 'military',                                
        'Sultanic Gatekeeper' : 'military',                                
        'Surgeon' : 'artisanal',                                            
        'Suyolcu' : 'artisanal',                                            
        'Sâhib-i timar' : 'military',                                      
        'Sürekçi' : 'artisanal',                                            
        'Sürsat Emîni' : 'artisanal',                                       
        'Süvari Mukabelecisi' : 'religious',                                
        'Sığırcı' : 'artisanal',                                            
        'Sığırtmaç (Cattle Herder)' : 'artisanal',
        'Tabbah' : 'artisanal',                                             
        'Tabbah (Cook)' : 'artisanal',                                      
        'Tabbâh (Cook)' : 'artisanal',                                      
        'Tabbâh-ı Sultânî' : 'artisanal',                                   
        'Tabbâh-ı Sultânî (Royal Cook)' : 'artisanal',                      
        'Tabbâh-ı hâs (Cook)' : 'artisanal',                                
        'Tabbâh-ı hâssa ' : 'artisanal',                                    
        'Tailor' : 'artisanal',                                            
        'Takyeci' : 'artisanal',                                            
        'Tanner' : 'artisanal',                                             
        'Tavern Keeper' : 'artisanal',                                      
        'Tavuk Kâtibi' : 'artisanal',                                       
        'Tavâşî' : 'artisanal',                                             
        'Tax Collector' : 'administrative',                                     
        'Teacher' : 'religious',                                           
        'Teacher and Scribe' : 'religious',                                 
        'Teberdar' : 'military',                                           
        'Teberdâr' : 'military',                                           
        'Teberdâr (Palace Guard)' : 'military',                            
        'Teberdâr-ı Hâssa' : 'military',                                   
        'Teberdâr-ı hâssa' : 'military',                                   
        'Tellâk' : 'artisanal',                                             
        'Tenant' : 'artisanal',                                             
        'Terekecibaşı' : 'artisanal',                                       
        'Termîmhâneci' : 'artisanal',                                       
        'Tersâne Emîni' : 'administrative',                                      
        'Tezkire-i evvel' : 'administrative',                                    
        'Tezkire-i sânî' : 'administrative',                                     
        'Tezkireci' : 'administrative',                                          
        'Tezkireci-i evvel' : 'administrative',                                  
        'Tezkireci-i sânî' : 'administrative',                                   
        'Timar Holder' : 'military',                                       
        'Timar holder' : 'military',                                       
        'Timarî' : 'military',                                             
        'Tinsmith' : 'artisanal',                                           
        'Tomb Keeper' : 'artisanal',                                        
        'Topçu' : 'military',                                              
        'Topçu (Artilleryman)' : 'military',                               
        'Topçular Bölükbaşısı' : 'military',                               
        'Topçuyân' : 'military',                                           
        'Town Crier' : 'artisanal',                                         
        'Trablus Defterdârı' : 'administrative',                                 
        'Trader/Merchant' : 'artisanal',                                 
        'Translator' : 'administrative',                                        
        'Treasurer' : 'administrative',                                         
        'Treasury Clerk' : 'administrative',                                     
        'Treasury Official' : 'administrative',                                  
        'Treasury official' : 'administrative',                                  
        'Trustee' : 'administrative',                                           
        'Trustee of Cafer Paşa Vakfı' : 'administrative',                        
        'Trustee of Gazi Ali Pasha\'s Waqf' : 'administrative',                   
        'Trustee of Kasım Paşa Vakfı' : 'administrative',                        
        'Trustee of the Waqf' : 'administrative',                                
        'Trustee of the waqf' : 'administrative',                                
        'Tâcir' : 'artisanal',                                              
        'Tüfenkçi' : 'military',                                           
        'Türbeci' : 'religious',                                           
        'Türbedâr' : 'religious',                                           
        'Tımar Sipahisi' : 'military',
        'Ulama' : 'religious',                                              
        'Ulufeci' : 'military',                                            
        'Ulufeci Başı' : 'military',                                       
        'Ulûfeci' : 'military',                                            
        'Ulûfeciler Cemâ‘atinden' : 'military',                            
        'Un Kapanı kethudâsı' : 'artisanal',                                
        'Vaiz (Preacher)' : 'religious',                                    
        'Vakf Clerk' : 'administrative',                                         
        'Vakf Trustee' : 'administrative',                                       
        'Vakf overseer' : 'administrative',                                      
        'Vakf scribe' : 'administrative',                                        
        'Vakf trustee' : 'administrative',                                       
        'Vekîl-i Harc' : 'administrative',                                       
        'Vekîl-i harc' : 'administrative',                                       
        'Vekîl-i mütevellî' : 'administrative',                                  
        'Vekîl-i Şer‘î' : 'administrative',                                      
        'Verenikos' : 'religious',                                          
        'Veznedar' : 'administrative',                                           
        'Vezîriazam' : 'administrative',                                         
        'Vidin Kadısı and Mukāta‘ât Nâzırı' : 'administrative',                  
        'Village Head' : 'administrative',                                       
        'Village Official' : 'administrative',                                   
        'Volunteer' : 'artisanal',                                          
        'Voynuklar çeribaşısı                               
        'Voyvoda' : 'administrative',                                            
        'Vâ‘iz' : 'religious',                                              
        'Waqf Clerk' : 'administrative',                                         
        'Waqf Collector' : 'administrative',                                     
        'Waqf Overseer' : 'administrative',                                      
        'Waqf Trustee' : 'administrative',                                      
        'Waqf trustee' : 'administrative',                                       
        'Water Carrier' : 'artisanal',                                      
        'Water Inspector' : 'administrative',                                    
        'Water Servant' : 'artisanal',                                      
        'Water carrier' : 'artisanal',                                      
        'Water overseer' : 'administrative',                                     
        'Waterway Division Head' : 'administrative',                             
        'Waterway Inspector' : 'administrative',                                 
        'Waterway Overseer' : 'administrative',                                  
        'Waterway Steward' : 'administrative',                                   
        'Waterway Supervisor' : 'administrative',                                
        'Waterway Worker' : 'artisanal',                                    
        'Witness' : 'administrative',                                          
        'Wool Merchant' : 'artisanal',                                      
        'Writer' : 'administrative',
        'Yapucı' : 'artisanal',                                             
        'Yasakçı' : 'military',                                            
        'Yava bekçisi' : 'military',                                       
        'Yaver' : 'military',                                              
        'Yaya' : 'military',                                              
        'Yayabaşı' : 'military',                                           
        'Yaycıbaşı' : 'military',                                          
        'Yazıcı' : 'artisanal',                                             
        'Yedekçi' : 'military',                                            
        'Yelkenci' : 'artisanal',                                           
        'Yesârî' : 'military',                                             
        'Yiğitbaşı' : 'military',                                          
        'Yogurt Seller' : 'artisanal',                                      
        'Yogurt seller' : 'artisanal',                                      
        'Yorgancı' : 'artisanal',                                           
        'Yâsin-hân' : 'religious',                                          
        'Za\'im' : 'military',                                              
        'Za\'im of Kerpiniş' : 'military',                                  
        'Zabitü’l-avâbık' : 'military',                                    
        'Zaim' : 'military',                                               
        'Zaîm' : 'military',                                               
        'Zaîmü’l-vakt' : 'military',                                       
        'Zağarcı' : 'military',                                            
        'Zağarcıbaşı' : 'military',                                        
        'Za‘îm' : 'military',                                             
        'Za‘îm of Üsküdar' : 'military',                                   
        'Za‘îm-i şehr' : 'military',                                       
        'Zevvâk' : 'artisanal',                                             
        'Zevvâk-ı Sultânî' : 'artisanal',                                   
        'Zevvâk-ı sultânî' : 'artisanal',                                   
        'Zevvâklar kethüdâsı' : 'artisanal',                                
        'Zimmi (Protected Person)                           
        'Zindancı' : 'military',                                           
        'Zindancı (Jailer)' : 'military',                                  
        'Zâbit' : 'military',                                              
        'Zâbıta' : 'military',                                             
        'Zâ‘îm Adam' : 'military',                                         
        'el-Münâdî' : 'administrative',                                          
        'el-muhzır' : 'administrative',                                          
        'Âbkeş' : 'military',                                              
        'Çavuş' : 'military',                                             
        'Çavuş-ı Alay' : 'military',                                       
        'Çavuş-ı Kapan-ı dakīk' : 'military',                              
        'Çavuş-ı bostan-ı hâssa' : 'military',                             
        'Çavuşbaşı' : 'military',                                          
        'Çavuşân Mirmîrân' : 'military',                                   
        'Çavuşân-ı Dergâh-ı Âlî' : 'military',                             
        'Çaşnigir Başı' : 'artisanal',                                      
        'Çaşnigîr' : 'artisanal',                                           
        'Çeribaşı' : 'artisanal',                                           
        'Çiftlik Kethüdası' : 'administrative',                                  
        'Çiçekçi (Florist)' : 'artisanal',                                  
        'Çorbacı' : 'artisanal',                                            
        'Çukacı' : 'military',                                             
        'Çukadar' : 'military',                                            
        'Çukadâr' : 'military',                                           
        'Çukadâr-ı sânî' : 'military',                                     
        'Çâşnigir' : 'artisanal',                                           
        'Çörekçi' : 'artisanal',                                            
        'Üsküdar Zaîmi' : 'military',                                      
        'Üsküdar Za‘îmi' : 'military',                                     
        'Üsküdar za‘îmi' : 'military',                                     
        'Üstâd-ı Bahçe-i Yamak                              
        'İbrahim Paşa Hamamcısı                             
        'İmam' : 'religious',                                              
        'İmâm' : 'religious',                                              
        'Şahincibaşı' : 'military',                                        
        'Şehrizol çavuş' : 'military',                                     
        'Şerbetçi' : 'artisanal',                                           
        'Şerbetçi (Sherbet Seller)' : 'artisanal',                          
        'Şeyh' : 'sufi',                                               
        'Şeyh-i Sakayân' : 'religious',                                     
        'Şeyhülislam' : 'religious',                                        
        'Şeyhülislâm' : 'religious',                                        
        'Şorbacı' : 'artisanal',                                            
        'Ştareri' : 'artisanal'                                          
    }
    
    return standardization_map.get(value, value)

In [1]:
# Check some examples of duplicates that should be the same
duplicates_check = (combined_df['cleaned_social_status']
                   .value_counts()
                   .sort_index())
print("Sample of values that need to be standardized:")
print(duplicates_check)

NameError: name 'combined_df' is not defined